In [127]:
import pandas as pd
import difflib
import datetime
import time
import math as math
from fuzzywuzzy import process

In [128]:
house = pd.read_csv('data/house_stocks.csv')
senate = pd.read_csv('data/senate_stocks.csv')
donations = pd.read_csv('data/congress_donations.csv')

In [129]:
#senate['transaction_date'] = pd.to_datetime(senate['transaction_date'])
senate.drop(columns = ['asset_type', 'comment', 'ptr_link'], inplace=True)
senate['representative'] = senate['senator']
senate.drop(columns = 'senator', inplace = True)
senate.loc[senate['type'] == 'Purchase', 'type'] = 'purchase'
senate.loc[senate['type'] == 'Sale (Full)', 'type'] = 'sale_full'
senate.loc[senate['type'] == 'Sale (Partial)', 'type'] = 'sale_partial'
senate.loc[senate['type'] == 'Exchange', 'type'] = 'exchange'
senate['timestamp'] = [int(datetime.datetime.strptime(timer,"%m/%d/%Y").timestamp()) for timer in senate['transaction_date']]

In [130]:
#house['transaction_date'] = pd.to_datetime(house['transaction_date'])
house.drop(columns = ['disclosure_year', 'disclosure_date', 'district', 'ptr_link', 'cap_gains_over_200_usd'], inplace = True)
house['representative'] = [rep[5:] for rep in house['representative']]
house['timestamp'] = [int(datetime.datetime.strptime(timer,"%Y-%m-%d").timestamp()) for timer in house['transaction_date']]

In [131]:
donations['party'] = [name[-2] for name in donations['Name']]
donations.columns = donations.columns.str.lower().str.replace(' ','_')
donations['name'] = [name[:-4] for name in donations['name']]

In [132]:
def clean_name(name):
    first = name.split()[0]
    second = name.split()[1:]
    first_rev = first[::-1]
    for i, let in enumerate(first_rev):
        if let == let.upper():
            index = i
            break
    for word in second:
        if len(word) < 3 or '\.' in word:
            second.remove(word)
    first = first[-(i+1):]
    second.insert(0,first)
    new = ' '.join(second)
    return new
    

In [133]:
def clean_name_rep(name):
    second = name.split()
    for word in second:
        if len(word) < 3 or '\.' in word:
            second.remove(word)
    new = ' '.join(second)
    return new

In [134]:
donations['name'] = [clean_name(name) for name in donations['name']]

In [135]:
pd.merge(left = house, right = donations, how = 'inner', left_on = 'representative', right_on = 'name')

,transaction_date,owner,ticker,asset_description,type,amount,representative,timestamp,name,total_raised,total_from_small_donors,percent_from_small_donors*,total_money_raised,party
0,2020-02-24,joint,BHC,Bausch Health Companies Inc.,purchase,"$1,001 - $15,000",Susie Lee,1582520400,Susie Lee,$4748019,$284775,6.00%,Raised over $100k,D
1,2020-03-06,joint,BAH,Booz Allen Hamilton Holding Corporation,sale_full,"$1,001 - $15,000",Susie Lee,1583470800,Susie Lee,$4748019,$284775,6.00%,Raised over $100k,D
2,2020-03-23,joint,BYD,Boyd Gaming Corporation,sale_full,"$1,001 - $15,000",Susie Lee,1584936000,Susie Lee,$4748019,$284775,6.00%,Raised over $100k,D
3,2020-02-04,joint,CTLT,"Catalent, Inc.",purchase,"$1,001 - $15,000",Susie Lee,1580792400,Susie Lee,$4748019,$284775,6.00%,Raised over $100k,D
4,2020-03-18,joint,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,1584504000,Susie Lee,$4748019,$284775,6.00%,Raised over $100k,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5140,2021-04-22,self,--,"MDLIVE, INC",sale_full,"$50,001 - $100,000",Mario Diaz-Balart,1619064000,Mario Diaz-Balart,$1590134,$2606,0.16%,Raised over $100k,R
5141,2020-07-14,--,TSLA,"Tesla, Inc.",sale_partial,"$15,001 - $50,000",Thomas Massie,1594699200,Thomas Massie,$1457316,$426531,29.27%,Raised over $100k,R
5142,2020-03-18,dependent,DIS,Walt Disney Company,purchase,"$15,001 - $50,000",Justin Amash,1584504000,Justin Amash,$1001120,$545077,54.45%,Raised over $100k,L
5143,2020-03-18,dependent,DIS,Walt Disney Company,purchase,"$1,001 - $15,000",Justin Amash,1584504000,Justin Amash,$1001120,$545077,54.45%,Raised over $100k,L


In [136]:
df = pd.concat([house, senate])
df.drop(columns = ['owner'], inplace = True)
df

,transaction_date,ticker,asset_description,type,amount,representative,timestamp
0,2020-02-24,BHC,Bausch Health Companies Inc.,purchase,"$1,001 - $15,000",Susie Lee,1582520400
1,2020-03-06,BAH,Booz Allen Hamilton Holding Corporation,sale_full,"$1,001 - $15,000",Susie Lee,1583470800
2,2020-03-23,BYD,Boyd Gaming Corporation,sale_full,"$1,001 - $15,000",Susie Lee,1584936000
3,2020-02-04,CTLT,"Catalent, Inc.",purchase,"$1,001 - $15,000",Susie Lee,1580792400
4,2020-03-18,CBRE,CBRE Group Inc Common Stock Class A,sale_partial,"$1,001 - $15,000",Susie Lee,1584504000
...,...,...,...,...,...,...,...
8551,08/17/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,Sheldon Whitehouse,1345176000
8552,08/16/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,Pat Roberts,1345089600
8553,08/15/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,Robert J Portman,1345003200
8554,08/02/2012,NaN,This filing was disclosed via scanned PDF. Use...,NaN,Unknown,Thomas R Carper,1343880000


In [137]:
df.dropna(inplace=True)
df.drop(df[df.ticker.str.contains('-')].index, inplace = True)
df.drop(df[df.ticker.str.contains('\.')].index, inplace = True)

In [138]:
df['representative'] = [clean_name_rep(name) for name in df['representative']]

In [139]:
dfmerge = pd.merge(left = df, right = donations, how = 'outer', left_on = 'representative', right_on = 'name')

In [143]:
df.to_csv('data/senate_and_house.csv')